<a href="https://colab.research.google.com/github/martakudt/Entregas/blob/main/transf_vs_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Bidirectional LDTM on IMDB**

LSTM, que significa Long Short-Term Memory (Memoria a Largo y Corto Plazo), es un tipo especial de red neuronal recurrente (RNN, por sus siglas en inglés) en Machine Learning, diseñada específicamente para procesar secuencias de datos y abordar el problema del gradiente desaparecido que afecta a las RNN tradicionales.

Las RNN son buenas para procesar datos secuenciales (como texto, series temporales o video), pero tienen dificultad para aprender dependencias a largo plazo debido a que, durante el proceso de entrenamiento, los gradientes que se propagan hacia atrás a través del tiempo tienden a disminuir (o aumentar) exponencialmente. Esto provoca que las RNN tradicionales olviden la información importante en secuencias largas.

Las LSTM solucionan este problema introduciendo un mecanismo de memoria que permite mantener información importante durante períodos largos, lo que las hace ideales para tareas donde las dependencias a largo plazo son clave.

In [2]:
import numpy as np
import keras
from keras import layers

max_features = 20000 # Solo consideramos las primeras 20 mil palabras
maxlen = 200 # Solo consideramos las primeras 200 palabras de cada review

In [4]:
# Entrada para secuencias de números enteros de longitud variable
inputs = keras.Input(shape=(None,), dtype="int32")
# Embebe cada entero en un vector de tamaño 128
x = layers.Embedding(max_features, 128)(inputs)
# Añadimos 2 capas Bidireccionales
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Añadimos una capa densa
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 128)           │       2,560,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, None, 128)           │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,757,761 (10.52 MB)

 Trainable params: 2,757,761 (10.52 MB)

 Non-trainable params: 0 (0.00 B)

Las capas bidireccionales en redes LSTM permiten procesar secuencias en ambas direcciones: hacia adelante y hacia atrás, utilizando dos LSTM separadas cuyos resultados se combinan. Esto mejora la capacidad del modelo para capturar dependencias tanto pasadas como futuras en la secuencia. En el código proporcionado, la primera capa bidireccional devuelve salidas completas de cada paso temporal (return_sequences=True) para preservar el contexto para capas posteriores, mientras que la segunda capa solo devuelve la última salida de la secuencia, condensando toda la información en un único vector.

## **Load the IMDB movie review sentiment data**

In [5]:
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words = max_features)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

# Usamos pad_sequence para estandarizar la longitud de la secuencia
# Esto truncará las secuencias de más de 200 palabras y rellenará con ceros las que sean más cortas
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
25000 Training sequences
25000 Validation sequences


In [6]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 456s 576ms/step - accuracy: 0.7565 - loss: 0.4876 - val_accuracy: 0.8514 - val_loss: 0.3532
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 506s 580ms/step - accuracy: 0.9107 - loss: 0.2301 - val_accuracy: 0.8538 - val_loss: 0.4321
